# HarvestStat Data Profiling - Guinea

In [1]:
import glob, json
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from tools import save_npz, PrintAdminUnits, PlotAdminShapes
from tools import FDW_PD_Sweeper, FDW_PD_AvalTable, FDW_PD_Compiling, FDW_PD_ValidateFnidName
from tools import FDW_PD_CreateAdminLink, FDW_PD_RatioAdminLink, FDW_PD_ConnectAdminLink
from tools import FDW_PD_CaliSeasonYear
from tools_graphic import PlotBarProduction, PlotLinePAY
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None

In [2]:
# Load FEWS NET administrative boundaries ------- #
epsg = 'epsg:32628' # Guinea
fn_shapes = sorted(glob.glob('../data/shapefile/fewsnet/GN_Admin?_????.shp'))
shape_all = []
for fn in fn_shapes:
    name = fn[-18:-4]
    exec('%s = gpd.read_file("%s").to_crs("%s")' % (name, fn, epsg))
    exec('%s["area"] = %s["geometry"].area/10**6' % (name, name))
    exec('shape_all.append(%s)' % (name))
shape_all = pd.concat(shape_all, axis=0).reset_index(drop=True)
PrintAdminUnits(shape_all)
# ----------------------------------------------- #

# FDW API host address -------------------------- #
host = 'https://fdw.fews.net'
auth = tuple(json.loads(open('token.json', "r").read()))
parameters = {
    'format': 'json',
    'country': 'Guinea',
    'product':['R011','R012','R013','R014','R015','R017','R018'],
    'survey_type': 'crop:best'
}
endpoint = '/api/cropproductionindicatorvalue/'
response = requests.get(host + endpoint, auth=auth, params=parameters, proxies={})
response.raise_for_status()
df = pd.DataFrame.from_records(response.json())
# ----------------------------------------------- #

- FEWS NET admin shapefiles ------------------- #
| year	 | Admin1   | # units   | Admin2   | # units   | Admin3   | # units   |
| 1990	 | GN1990A1 | 8	| GN1990A2	| 34	| nan	| 0	|
| 2014	 | GN2014A1 | 8	| GN2014A2	| 34	| nan	| 0	|
----------------------------------------------- #


In [3]:
# Manual Pre-processing before Sweeping --------- #
# 1. Default setting 
# a) None-type crop production system
df.loc[df['crop_production_system'].isna(), 'crop_production_system'] = 'none'
df.loc[df['crop_production_system'] == '', 'crop_production_system'] = 'none'
# b) None-type population group
df.loc[df['population_group'].isna(), 'population_group'] = 'none'
df.loc[df['population_group'] == '', 'population_group'] = 'none'
# Replace season name
df['season_name'] = df['season_name'].replace({
    'Main harvest': "Main"
})
# Remove admin-1 level data
df = df[~(df['fnid'].apply(lambda x: x[6:8] == 'A1') & (df['season_date'].isin(['2009-09-01', '2008-09-01'])))]
# ----------------------------------------------- #

# FDW Production Data Inspection ---------------- #
df, df_raw = FDW_PD_Sweeper(df, area_priority='Area Planted')
table_dict = FDW_PD_AvalTable(df, shape_all)
# ----------------------------------------------- #

# FEWS NET Shapefile comparison ----------------- #
shape_used = pd.concat([
    GN_Admin1_1990, GN_Admin2_1990
], axis=0)
PlotAdminShapes(shape_used, label=True)
# ----------------------------------------------- #

- Remove missing records ---------------------- #
Orignial data points: 780
780/780 "Quantity Produced" points are retained.
Current data points: 780

- Minor changes are applied ------------------- #

- Basic information --------------------------- #
Data period: 2010 - 2015
4 grain types are found: Cassava, Groundnuts (In Shell), Maize (Corn), Rice (Paddy)
1 seasons are found: Main (09-01)
1 crop production system are found: none
Data sources include:
[1] L'Agence Nationale des Statistiques Agricoles et Alimentaires, Guinea --- Dynamique de production, Guinea
Administrative-1 fnids: 0
Administrative-2 fnids: 33
0 reporting units are found: 

- Total production over time ------------------ #
season_name             Main
                            
Cassava                31.5%
Groundnuts (In Shell)   9.8%
Maize (Corn)           20.2%
Rice (Paddy)           38.5%

- Crop calendar ------------------------------- #
                 product season_name planting_month harvest_month
0      

- Guinea crop seasonal calendar

![FEWS NET](../figures/crop_calendar/seasonal-calendar-guinea.png)

![USDA](https://ipad.fas.usda.gov/rssiws/al/crop_calendar/images/wafrica_gv_calendar.png)

- FDW data consists of `GN1990A1` and `GN1990A2`.

| Year | Admin-1 | # units  | Admin-2  | # units |
| :---: | :----:  | :----:   | :----:   | :---:  |
| 1990	 | **`GN1990A1`** | 8	| **`GN1990A2`**	| 34	|
| 2014	 | GN2014A1 | 8	| GN2014A2	| 34	|

- Comparison between admin boundaries.

![image](../figures/GN_admin_shapes.png)

- **FDW data consists of only "Quantity Produced" for 8 years**.
- Different admin-level for two periods: GN1990A1 (2008-2009) and GN1990A2 (2010-2015)
- **`GN1990A2`** is used to represent current admin-level 2 crop data.
- Guinea has a single crop seasons: Main harvest (09-01).
- Guinea has no crop production system.
- Guinea has no population group.

In [4]:
# Define the latest shapefile ------------------- #
latest_level = 2
shape_latest = GN_Admin2_1990.copy().to_crs('epsg:4326')
# ----------------------------------------------- #

# Validation of FNIDs and Names ----------------- #
df = FDW_PD_ValidateFnidName(df, shape_used, shape_latest)
# ----------------------------------------------- #

# FDW Production Data Compiling ----------------- #
area_new, prod_new = FDW_PD_Compiling(df, shape_used)
# ----------------------------------------------- #

In [5]:
# Manual correction ----------------------------- #
crop_new = prod_new/area_new
# ----------------------------------------------- #

# Complete long format DataFrame ---------------- #
df_area = area_new.T.stack().rename('value').reset_index()
df_area['indicator'] = 'area'
df_prod = prod_new.T.stack().rename('value').reset_index()
df_prod['indicator'] = 'production'
df_yield = (prod_new/area_new).T.stack().rename('value').reset_index()
df_yield['indicator'] = 'yield'
stack = pd.concat([df_area, df_prod, df_yield], axis=0)
# Add "planting year"
cols = ['season_name','product','crop_production_system','planting_month','harvest_year','harvest_month','planting_year']
season_table = df[cols].drop_duplicates()
stack = stack.merge(season_table, on=cols[:-1])
# Add country and admin names
stack = stack.merge(df[['fnid','country','country_code','admin_1','admin_2']].drop_duplicates(), on='fnid', how='inner')
names = [
    'fnid','country','country_code','admin_1','admin_2','name',
    'product','season_name','planting_year','planting_month','harvest_year','harvest_month',
    'crop_production_system','indicator','value'
]
stack = stack[names]
# ----------------------------------------------- #

# Final Processing ------------------------------ #
# Manual change of product name
product_category_custom = {
    'Cassava': 'Cassava', 
    'Groundnuts (In Shell)': 'Groundnuts',
    'Maize (Corn)': 'Maize', 
    'Rice (Paddy)': 'Rice',
}
stack['product'] = stack['product'].replace(product_category_custom)
# Calibration of crop calendar using External Crop Calendar (ECC)
ecc = pd.read_csv('../data/crop_calendar/external_crop_calendar.csv')
ecc = ecc[ecc['country'] == 'Guinea']
stack, link_ratio = FDW_PD_CaliSeasonYear(stack, ecc, link_ratio=None)
# None admin names
stack.loc[stack['admin_1'].isna(), 'admin_1'] = 'none'
stack.loc[stack['admin_2'].isna(), 'admin_2'] = 'none'
# ----------------------------------------------- #

# Save data
fn_out = '../data/crop/adm_crop_production_GN.csv'
stack.to_csv(fn_out); print(f'{fn_out} is saved.')


All [season_name, product, crop_production_system] are in the external crop calendar.
../data/crop/adm_crop_production_GN.csv is saved.


## Visualization of production data

In [6]:
# Bar chart of national crop production
country_iso, country_name = 'GN', 'Guinea'
df = pd.read_csv('../data/crop/adm_crop_production_%s.csv' % country_iso, index_col=0)
df['year'] = df['harvest_year']
year = [df['year'].min(), df['year'].max()]
product_order = ['Cassava','Groundnuts','Rice', 'Maize']
for season_name in ['Main']:
    footnote = 'National crop production in %s - %s' % (country_name, season_name)
    fn_save = '../figures/%s_bar_natgrainprod_%s.png' % (country_iso, season_name)
    sub = df[df['season_name'] == season_name]
    fig = PlotBarProduction(sub, year, product_order, footnote, fn_save)
    fig.show()

../figures/GN_bar_natgrainprod_Main.png is saved.


In [7]:
# Lineplot of Production-Area-Yield time-series
country_iso, country_name = 'GN', 'Guinea'
df = pd.read_csv('../data/crop/adm_crop_production_%s.csv' % country_iso, index_col=0)
df['year'] = df['harvest_year']
year = [df['year'].min(), df['year'].max()]
product_season = [
    ['Cassava','Main'],
    ['Groundnuts','Main'],
    ['Maize','Main'],
    ['Rice','Main'],
]
for product_name, season_name in product_season:
    footnote = 'Production-Area-Yield time-series of %s - %s - %s' % (country_iso, product_name, season_name)
    fn_save = '../figures/%s_line_pay_%s_%s.png' % (country_iso, product_name, season_name)
    sub = df[(df['product'] == product_name) & (df['season_name'] == season_name)]
    fig = PlotLinePAY(sub, year, footnote, fn_save)
    fig.show()

../figures/GN_line_pay_Cassava_Main.png is saved.


../figures/GN_line_pay_Groundnuts_Main.png is saved.


../figures/GN_line_pay_Maize_Main.png is saved.


../figures/GN_line_pay_Rice_Main.png is saved.
